In [1]:
# Import necessary libraries
import pandas as pd
import datetime as dt
from lifetimes import BetaGeoFitter, GammaGammaFitter
from lifetimes.utils import summary_data_from_transaction_data

In [2]:
# importing the data
df = pd.read_excel(r"C:\Users\vishal\OneDrive\Desktop\case study\online+retail\Online Retail.xlsx")
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [3]:
df.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [4]:
# Data Cleaning
df.dropna(subset=['CustomerID'], inplace=True)
df = df[df['Quantity'] > 0]
df = df[df['UnitPrice'] > 0]

In [5]:
# Create 'TotalPrice' column
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

In [6]:
# Convert 'InvoiceDate' to datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [7]:
# Set analysis date as one day after the last transaction
analysis_date = df['InvoiceDate'].max() + dt.timedelta(days = 1)

In [8]:
# Prepare data for BG/NBD and Gamma-Gamma models
summary = summary_data_from_transaction_data(df, 'CustomerID', 'InvoiceDate', monetary_value_col='TotalPrice', observation_period_end=analysis_date)

In [9]:
# Fit the BG/NBD model
bgf = BetaGeoFitter(penalizer_coef=0.0)
bgf.fit(summary['frequency'], summary['recency'], summary['T'])

<lifetimes.BetaGeoFitter: fitted with 4338 subjects, a: 0.00, alpha: 69.23, b: 6.65, r: 0.83>

In [10]:
# Predict future purchases
summary['predicted_purchases'] = bgf.conditional_expected_number_of_purchases_up_to_time(12, summary['frequency'], summary['recency'], summary['T'])

In [11]:
print(summary[summary['monetary_value'] <= 0])

            frequency  recency      T  monetary_value  predicted_purchases
CustomerID                                                                
12346.0           0.0      0.0  326.0             0.0             0.025068
12349.0           0.0      0.0   19.0             0.0             0.112286
12350.0           0.0      0.0  311.0             0.0             0.026057
12353.0           0.0      0.0  205.0             0.0             0.036129
12354.0           0.0      0.0  233.0             0.0             0.032782
...               ...      ...    ...             ...                  ...
18276.0           0.0      0.0   44.0             0.0             0.087496
18277.0           0.0      0.0   59.0             0.0             0.077262
18278.0           0.0      0.0   74.0             0.0             0.069171
18280.0           0.0      0.0  278.0             0.0             0.028533
18281.0           0.0      0.0  181.0             0.0             0.039594

[1548 rows x 5 columns]


In [12]:
summary = summary[summary['monetary_value'] > 0]

In [13]:
# Fit the Gamma-Gamma model
ggf = GammaGammaFitter(penalizer_coef=0.0)
ggf.fit(summary['frequency'], summary['monetary_value'])

<lifetimes.GammaGammaFitter: fitted with 2790 subjects, p: 2.10, q: 3.45, v: 485.89>

In [14]:
# Predict average monetary value
summary['predicted_monetary_value'] = ggf.conditional_expected_average_profit(summary['frequency'], summary['monetary_value'])

In [15]:
# Calculate CLV
summary['CLV'] = ggf.customer_lifetime_value(bgf, summary['frequency'], summary['recency'], summary['T'], summary['monetary_value'],
                                            time = 12,
                                            discount_rate = 0.01)

In [16]:
print(summary[['predicted_purchases', 'predicted_monetary_value', 'CLV']])

            predicted_purchases  predicted_monetary_value          CLV
CustomerID                                                            
12347.0                0.187261                569.978836  3000.838905
12348.0                0.107098                333.784235  1005.171489
12352.0                0.223486                376.175359  2363.309727
12356.0                0.085738                324.039419   781.226541
12358.0                0.099413                539.907126  1508.872854
...                         ...                       ...          ...
18272.0                0.220970                474.368524  2946.432180
18273.0                0.103563                201.838133   587.706959
18282.0                0.111567                260.340479   816.446938
18283.0                0.407289                174.532812  1998.132071
18287.0                0.124906                492.169257  1728.192616

[2790 rows x 3 columns]
